<a href="https://colab.research.google.com/github/nikhil-xyz/Natural_Language_Processing/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikhil-xyz/Natural_Language_Processing.git

Cloning into 'Natural_Language_Processing'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 7), reused 19 (delta 4), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Natural_Language_Processing/data/Fake_News_Classifier/Fake_news.csv')

In [4]:
df = df.dropna()

In [5]:
X = df.drop('Label', axis=1)
y = df['Label']

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [8]:
vocab_size = 5000

In [9]:
messages = X.copy()
messages.reset_index(inplace=True)

In [10]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['Body'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [13]:
onehot_rep = [one_hot(words, vocab_size) for words in corpus]

In [14]:
sentence_len = 2000

In [15]:
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sentence_len)

In [16]:
len(embedded_docs)

3988

In [17]:
embedding_vector_features = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sentence_len))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
import numpy as np
X = np.array(embedded_docs)
y = np.array(y)

In [20]:
X.shape

(3988, 2000)

In [21]:
y.shape

(3988,)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [23]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/10
100/100 [==============================] - 556s 6s/step - loss: 0.2687 - accuracy: 0.8950 - val_loss: 0.0865 - val_accuracy: 0.9687
Epoch 2/10
100/100 [==============================] - 554s 6s/step - loss: 0.0411 - accuracy: 0.9897 - val_loss: 0.0653 - val_accuracy: 0.9812
Epoch 3/10
100/100 [==============================] - 549s 5s/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.0801 - val_accuracy: 0.9724
Epoch 4/10
100/100 [==============================] - 547s 5s/step - loss: 0.0129 - accuracy: 0.9947 - val_loss: 0.1105 - val_accuracy: 0.9637
Epoch 5/10
100/100 [==============================] - 547s 5s/step - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.1064 - val_accuracy: 0.9749
Epoch 6/10
100/100 [==============================] - 543s 5s/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.1014 - val_accuracy: 0.9737
Epoch 7/10
100/100 [==============================] - 542s 5s/step - loss: 5.4577e-04 - accuracy: 1.0000 - val_loss: 0.1100 - val_accuracy: 0.

In [71]:
y_pred = model.predict(X_test)

25/25 [==============================] - 37s 1s/step


In [73]:
y_pred[:5]

array([[4.0173341e-05],
       [7.6678982e-05],
       [9.9999058e-01],
       [5.9525075e-05],
       [4.5091831e-05]], dtype=float32)

In [86]:
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])):
    if y_pred[i][j] > 0.5:
      y_pred[i][j] = int(1)
    else:
      y_pred[i][j] = int(0)

In [87]:
y_pred[:5]

array([[0.],
       [0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [76]:
y_test[0][:5]

array([0, 0, 1, 0, 0])

In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

ValueError: ignored